Same as exp_1 but:
1. with LayerNorms
2. init constants [0., -2.]
4. Order of activation, normalization is changed to normalization -> activation

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../src')

In [8]:
import torch
from torch.utils.data import DataLoader

from flow_regressors.flow_dataset import FlowDataset
from flow_regressors.regressors import DensePoseRegressorV3

from training.train_flow import train
from training.loss_pose import LossPose

from tqdm.auto import tqdm

### 1. Data

In [4]:
train_data = FlowDataset(
    root_path='/home/project/data_sample/flow/train',
    intrinsics_path='/home/project/ScanNet/scannet_indices/intrinsics.npz'
    )

train_loader = DataLoader(train_data, batch_size=128, shuffle=True, drop_last=True, pin_memory=True, num_workers=0)

val_data = FlowDataset(
    root_path='/home/project/data_sample/flow/val',
    intrinsics_path='/home/project/ScanNet/scannet_indices/intrinsics.npz'
    )

val_loader = DataLoader(val_data, batch_size=128, shuffle=False, drop_last=False, pin_memory=True, num_workers=0)

### 2. Configuration

In [5]:
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

config = dict(
    general = dict(
        experiment_name='FLOW_TO_POSE_EXP_9',
        device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu'),
        
        n_epochs=10,
        n_steps_per_epoch=len(train_loader.dataset) // train_loader.batch_size,
        n_accum_steps=1,
        batch_size=train_loader.batch_size,
        
        swa=False,
        n_epochs_swa=None,
        n_steps_between_swa_updates=None,
          
        repeat_val_epoch=1,
        repeat_save_epoch=5,
          
        model_save_path='../src/weights/FLOW_TO_POSE_EXP_9'
    )
)

### 3. Model

In [9]:
# Inference on (120, 160) input:

model = DensePoseRegressorV3(init_loss_weights=[0., -2.], dropout=None)
model.to(config['general']['device']);

In [7]:
# data = train_data[0]
# data['flow'] = torch.from_numpy(data['flow']).cuda().unsqueeze(0) 
# data['K_q'] = torch.from_numpy(data['K_q']).cuda().unsqueeze(0) 
# data['K_s'] = torch.from_numpy(data['K_s']).cuda().unsqueeze(0) 
# model.eval();

In [8]:
# %%timeit

# with torch.no_grad():
#     q, t = model(
#         data['flow'], data['K_q'], data['K_s'], (0.25, 0.25), (0.25, 0.25)
#        )

### 4. Loss, optimizer, scheduler

In [10]:
train_loss = LossPose(agg_type='mean')
val_loss = LossPose()

In [11]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-6)
config['optimizer'] = optimizer.__dict__['defaults']

In [12]:
scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                   step_size=2,
                   gamma=0.8)

### 6. Experiment

In [13]:
train(model, optimizer, scheduler, train_loss, val_loss, train_loader, val_loader, config, **config['general'])

wandb: Currently logged in as: kovanic. Use `wandb login --relogin` to force relogin


  2%|██▎                                                                                                                   | 15/775 [00:15<13:03,  1.03s/it]


Train batch loss,█▇▇▆▆▆▅▅▅▄▃▂▂▁▁
Train batch loss,1.46079


KeyboardInterrupt: 